## CH 03 - Creating and Persisting DataFrames

In [46]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 10, 'display.max_rows', 10)

### How to do it\...

In [2]:
fname = ['Paul', 'John', 'Richard', 'George']
lname = ['McCartney', 'Lennon', 'Starkey', 'Harrison']
birth = [1942, 1940, 1940, 1943]

In [3]:
# Create a dictionary from the lists, mapping the column name to the list:

people = {'first': fname, 'last': lname, 'birth': birth}

In [4]:
# Create a DataFrame from the dictionary:

beatles = pd.DataFrame(people)
beatles

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


### How it works\...

In [5]:
beatles.index

RangeIndex(start=0, stop=4, step=1)

In [6]:
pd.DataFrame(people, index=['a', 'b', 'c', 'd'])

,first,last,birth
a,Paul,McCartney,1942
b,John,Lennon,1940
c,Richard,Starkey,1940
d,George,Harrison,1943


### There\'s More

In [7]:
# You can also create a DataFrame from a list of dictionaries:

pd.DataFrame(
[{"first":"Paul","last":"McCartney", "birth":1942},
 {"first":"John","last":"Lennon", "birth":1940},
 {"first":"Richard","last":"Starkey", "birth":1940},
 {"first":"George","last":"Harrison", "birth":1943}])

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [8]:
[{"first":"Paul","last":"McCartney", "birth":1942},
 {"first":"John","last":"Lennon", "birth":1940},
 {"first":"Richard","last":"Starkey", "birth":1940},
 {"first":"George","last":"Harrison", "birth":1943}],

columns=(['last', 'first', 'birth'])

### How to do it\...

In [9]:
beatles

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [10]:
# Write the DataFrame to a CSV file

from io import StringIO
fout = StringIO()
beatles.to_csv(fout)  # use a filename instead of fout

In [11]:
print(fout.getvalue())

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943



### There\'s More

In [12]:
_ = fout.seek(0)
pd.read_csv(fout)

,Unnamed: 0,first,last,birth
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940
2,2,Richard,Starkey,1940
3,3,George,Harrison,1943


In [13]:
_ = fout.seek(0)
pd.read_csv(fout, index_col=0)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [14]:
fout = StringIO()
beatles.to_csv(fout, index=False) 
print(fout.getvalue())

first,last,birth
Paul,McCartney,1942
John,Lennon,1940
Richard,Starkey,1940
George,Harrison,1943



### How to do it\...

In [15]:
# Reading large CSV files

diamonds = pd.read_csv('../data/diamonds.csv', nrows=1000)
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
995,0.54,Ideal,D,VVS2,61.4,52.0,2897,5.30,5.34,3.26
996,0.72,Ideal,E,SI1,62.5,55.0,2897,5.69,5.74,3.57
997,0.72,Good,F,VS1,59.4,61.0,2897,5.82,5.89,3.48
998,0.74,Premium,D,VS2,61.8,58.0,2897,5.81,5.77,3.58


In [16]:
# Use the .info method to see how much memory the sample of data uses:

diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   float64
 1   cut      1000 non-null   object 
 2   color    1000 non-null   object 
 3   clarity  1000 non-null   object 
 4   depth    1000 non-null   float64
 5   table    1000 non-null   float64
 6   price    1000 non-null   int64  
 7   x        1000 non-null   float64
 8   y        1000 non-null   float64
 9   z        1000 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 78.3+ KB


In [17]:
# Use the dtype parameter to read_csv to tell it to use the correct (or smaller) numeric types:

diamonds2 = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'x': np.float32,
           'y': np.float32, 'z': np.float32,
           'price': np.int16})

In [18]:
# By changing the numeric types, we use about 62% of the memory.

diamonds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   float32
 1   cut      1000 non-null   object 
 2   color    1000 non-null   object 
 3   clarity  1000 non-null   object 
 4   depth    1000 non-null   float32
 5   table    1000 non-null   float32
 6   price    1000 non-null   int16  
 7   x        1000 non-null   float32
 8   y        1000 non-null   float32
 9   z        1000 non-null   float32
dtypes: float32(6), int16(1), object(3)
memory usage: 49.0+ KB


In [19]:
# Use the dtype parameter to use change object types to categoricals. First, inspect
# the .value_counts method of the object columns. If they are low cardinality, you
# can convert them to categorical columns to save even more memory:

diamonds.describe()

,carat,depth,table,price,x,y,z
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,0.689280,61.722800,57.734700,2476.54000,5.605940,5.599180,3.457530
std,0.195291,1.758879,2.467946,839.57562,0.625173,0.611974,0.389819
min,0.200000,53.000000,52.000000,326.00000,3.790000,3.750000,2.270000
25%,0.700000,60.900000,56.000000,2777.00000,5.640000,5.630000,3.450000
50%,0.710000,61.800000,57.000000,2818.00000,5.770000,5.760000,3.550000
75%,0.790000,62.600000,59.000000,2856.00000,5.920000,5.910000,3.640000
max,1.270000,69.500000,70.000000,2898.00000,7.120000,7.050000,4.330000


In [20]:
diamonds2.describe()

,carat,depth,table,price,x,y,z
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,0.689280,61.722801,57.734699,2476.54000,5.605940,5.599180,3.457530
std,0.195291,1.758878,2.467944,839.57562,0.625173,0.611972,0.389819
min,0.200000,53.000000,52.000000,326.00000,3.790000,3.750000,2.270000
25%,0.700000,60.900002,56.000000,2777.00000,5.640000,5.630000,3.450000
50%,0.710000,61.799999,57.000000,2818.00000,5.770000,5.760000,3.550000
75%,0.790000,62.599998,59.000000,2856.00000,5.920000,5.910000,3.640000
max,1.270000,69.500000,70.000000,2898.00000,7.120000,7.050000,4.330000


In [21]:
diamonds2.cut.value_counts()

cut
Ideal        333
Premium      290
Very Good    226
Good          89
Fair          62
Name: count, dtype: int64

In [22]:
diamonds2.color.value_counts()

color
E    240
F    226
G    139
D    129
H    125
I     95
J     46
Name: count, dtype: int64

In [23]:
diamonds2.clarity.value_counts()

clarity
SI1     306
VS2     218
VS1     159
SI2     154
VVS2     62
VVS1     58
I1       29
IF       14
Name: count, dtype: int64

In [24]:
diamonds3 = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'x': np.float32,
           'y': np.float32, 'z': np.float32,
           'price': np.int16,
           'cut': 'category', 'color': 'category',
           'clarity': 'category'})

In [25]:
diamonds3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float32 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float32 
 5   table    1000 non-null   float32 
 6   price    1000 non-null   int16   
 7   x        1000 non-null   float32 
 8   y        1000 non-null   float32 
 9   z        1000 non-null   float32 
dtypes: category(3), float32(6), int16(1)
memory usage: 29.4 KB


In [26]:
np.iinfo(np.int8)

iinfo(min=-128, max=127, dtype=int8)

In [27]:
np.finfo(np.float16)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

In [28]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']
diamonds4 = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'price': np.int16,
           'cut': 'category', 'color': 'category',
           'clarity': 'category'},
    usecols=cols)

In [29]:
diamonds4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float32 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float32 
 5   table    1000 non-null   float32 
 6   price    1000 non-null   int16   
dtypes: category(3), float32(3), int16(1)
memory usage: 17.6 KB


In [30]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']
diamonds_iter = pd.read_csv('../data/diamonds.csv', nrows=1000,
    dtype={'carat': np.float32, 'depth': np.float32,
           'table': np.float32, 'price': np.int16,
           'cut': 'category', 'color': 'category',
           'clarity': 'category'},
    usecols=cols,
    chunksize=200)

In [31]:
def process(df):
    return f'processed {df.size} items'

In [32]:
for chunk in diamonds_iter:
    process(chunk)

### How it works\...

### There\'s more \...

In [33]:
diamonds.price.memory_usage()

8132

In [34]:
diamonds.price.memory_usage(index=False)

8000

In [35]:
diamonds.cut.memory_usage()

8132

In [36]:
diamonds.cut.memory_usage(deep=True)

63465

In [37]:
diamonds4.to_feather('../tmp/d.arr')
diamonds5 = pd.read_feather('../tmp/d.arr')

In [38]:
diamonds4.to_parquet('../tmp/d.pqt')

### How to do it\...

In [39]:
import xlwt

In [40]:
beatles.to_excel('../tmp/beat.xlsx')

In [41]:
beatles.to_excel('../tmp/beat.xlsx')

In [42]:
beat2 = pd.read_excel('../tmp/beat.xlsx')
beat2

,Unnamed: 0,first,last,birth
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940
2,2,Richard,Starkey,1940
3,3,George,Harrison,1943


In [43]:
beat2 = pd.read_excel('../tmp/beat.xlsx', index_col=0)
beat2

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [44]:
beat2.dtypes

first    object
last     object
birth     int64
dtype: object

### How it works\...

### There\'s more\...

In [47]:
# How to write files to Excel

from openpyxl import Workbook

In [48]:
xl_writer = pd.ExcelWriter('../tmp/beat.xlsx')
beatles.to_excel(xl_writer, sheet_name='All')
beatles[beatles.birth < 1941].to_excel(xl_writer, sheet_name='1940')

### How to do it\...

In [51]:
autos = pd.read_csv('../data/vehicles.csv.zip', low_memory=False)
autos

,barrels08,barrelsA08,charge120,charge240,city08,...,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
39096,14.982273,0.0,0.0,0.0,19,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
39097,14.330870,0.0,0.0,0.0,20,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
39098,15.695714,0.0,0.0,0.0,18,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
39099,15.695714,0.0,0.0,0.0,18,...,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [50]:
autos.modifiedOn.dtype

dtype('O')

In [ ]:
autos.modifiedOn

In [ ]:
pd.to_datetime(autos.modifiedOn)  # doctest: +SKIP

In [ ]:
autos = pd.read_csv('data/vehicles.csv.zip',
    parse_dates=['modifiedOn'])  # doctest: +SKIP
autos.modifiedOn

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('data/kaggle-survey-2018.zip') as z:
    print('\n'.join(z.namelist()))
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    kag_questions = kag.iloc[0]
    survey = kag.iloc[1:]

In [ ]:
print(survey.head(2).T)

### How it works\...

### There\'s more\...

### How to do it\...

In [ ]:
import sqlite3
con = sqlite3.connect('data/beat.db')
with con:
    cur = con.cursor()
    cur.execute("""DROP TABLE Band""")
    cur.execute("""CREATE TABLE Band(id INTEGER PRIMARY KEY,
        fname TEXT, lname TEXT, birthyear INT)""")
    cur.execute("""INSERT INTO Band VALUES(
        0, 'Paul', 'McCartney', 1942)""")
    cur.execute("""INSERT INTO Band VALUES(
        1, 'John', 'Lennon', 1940)""")
    _ = con.commit()

In [ ]:
import sqlalchemy as sa
engine = sa.create_engine(
  'sqlite:///data/beat.db', echo=True)
sa_connection = engine.connect()

In [ ]:
beat = pd.read_sql('Band', sa_connection, index_col='id')
beat

In [ ]:
sql = '''SELECT fname, birthyear from Band'''
fnames = pd.read_sql(sql, con)
fnames

### How it work\'s\...

In [ ]:
import json
encoded = json.dumps(people)
encoded

In [ ]:
json.loads(encoded)

### How to do it\...

In [ ]:
beatles = pd.read_json(encoded)
beatles

In [ ]:
records = beatles.to_json(orient='records')
records

In [ ]:
pd.read_json(records, orient='records')

In [ ]:
split = beatles.to_json(orient='split')
split

In [ ]:
pd.read_json(split, orient='split')

In [ ]:
index = beatles.to_json(orient='index')
index

In [ ]:
pd.read_json(index, orient='index')

In [ ]:
values = beatles.to_json(orient='values')
values

In [ ]:
pd.read_json(values, orient='values')

In [ ]:
(pd.read_json(values, orient='values')
   .rename(columns=dict(enumerate(['first', 'last', 'birth'])))
)

In [ ]:
table = beatles.to_json(orient='table')
table

In [ ]:
pd.read_json(table, orient='table')

### How it works\...

### There\'s more\...

In [ ]:
output = beat.to_dict()
output

In [ ]:
output['version'] = '0.4.1'
json.dumps(output)

### How to do it\...

In [ ]:
url ='https://en.wikipedia.org/wiki/The_Beatles_discography'
dfs = pd.read_html(url)
len(dfs)

In [ ]:
dfs[0]

In [ ]:
url ='https://en.wikipedia.org/wiki/The_Beatles_discography'
dfs = pd.read_html(url, match='List of studio albums', na_values='—')
len(dfs)

In [ ]:
dfs[0].columns

In [ ]:
url ='https://en.wikipedia.org/wiki/The_Beatles_discography'
dfs = pd.read_html(url, match='List of studio albums', na_values='—',
    header=[0,1])
len(dfs)

In [ ]:
dfs[0]

In [ ]:
dfs[0].columns

In [ ]:
df = dfs[0]
df.columns = ['Title', 'Release', 'UK', 'AUS', 'CAN', 'FRA', 'GER',
    'NOR', 'US', 'Certifications']
df

In [ ]:
res = (df
  .pipe(lambda df_: df_[~df_.Title.str.startswith('Released')])
  .iloc[:-1]
  .assign(release_date=lambda df_: pd.to_datetime(
             df_.Release.str.extract(r'Released: (.*) Label')
               [0]
               .str.replace(r'\[E\]', '')
          ),
          label=lambda df_:df_.Release.str.extract(r'Label: (.*)')
         )
   .loc[:, ['Title', 'UK', 'AUS', 'CAN', 'FRA', 'GER', 'NOR',
            'US', 'release_date', 'label']]
)
res

### How it works\...

### There is more\...

In [ ]:
url = 'https://github.com/mattharrison/datasets/blob/master/data/anscombes.csv'
dfs = pd.read_html(url, attrs={'class': 'csv-data'})
len(dfs)

In [ ]:
dfs[0]